#### Preface

In this short tutorial, the tuning process for further evaluation is demonstrated. For this purpose, datasets from the [chunking_evaluation](https://github.com/brandonstarxel/chunking_evaluation) framework will be used.

#### Packages installation 

In [1]:
!pip install "git+https://github.com/panalexeu/horchunk.git"
!pip install requests
!pip install chroma
!pip install numpy

  Cloning https://github.com/panalexeu/horchunk.git to /tmp/pip-req-build-cygrr0x5
  Running command git clone --filter=blob:none --quiet https://github.com/panalexeu/horchunk.git /tmp/pip-req-build-cygrr0x5
  Resolved https://github.com/panalexeu/horchunk.git to commit 4fd7e6936057689b3677d92f7a1be398cbf388fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


#### Loading datasets from git

In [2]:
import requests 

links_dict = dict(
    wikitexts='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/wikitexts.md',
    chatlogs='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/chatlogs.md',
    finance='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/finance.md',
    pubmed='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/pubmed.md',
    state_of_the_union='https://raw.githubusercontent.com/brandonstarxel/chunking_evaluation/refs/heads/main/chunking_evaluation/evaluation_framework/general_evaluation_data/corpora/state_of_the_union.md'
) 


def load_datasets(links_dict: dict) -> dict:
    data = dict()
    for name, link in links_dict.items():
        response = requests.get(link)
        data[name] = response.text 
        
    return data 

datasets = load_datasets(links_dict)
for key in datasets.keys():
    print(f'{key}: {len(datasets[key].split())} words')

wikitexts: 22406 words
chatlogs: 5968 words
finance: 116860 words
pubmed: 75846 words
state_of_the_union: 8468 words


#### Tuning

Let's start by instantiating WindowTuner.

In [3]:
from horchunk.chunkers import WindowTuner
from chromadb.utils import embedding_functions

ef = embedding_functions.DefaultEmbeddingFunction() # all-MiniLM-L6-v2
tuner = WindowTuner(ef) 

/home/oleksii/python/semantic_chunking/semchunk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this tutorial, we will tune the minimum threshold for document of 3 sentences in size. We will repeat this process for every dataset. The identified thresholds will then be averaged to determine a single, generalized threshold value.

In [4]:
thresholds = []
DEPTH = 3

**wikitexts**

In [5]:
from horchunk.splitters import SentenceSplitter 

splitter = SentenceSplitter(text=datasets['wikitexts'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1090/1090 [01:06<00:00, 16.31it/s]


1090 chunks formed

Values range: [-0.022758223116397858 ... 1.0]

Steps to tune: 10

--------------------------------

...

chunk:  This ammunition , and that which I brought with me , was rapidly prepared for use at the Laboratory 
established at the Little Rock Arsenal for that purpose .As illustrating as the pitiful scarcity of material in the
country , the fact may be stated that it was found necessary to use public documents of the State Library for 
cartridge paper . <unk> were employed or conscripted , tools purchased or impressed , and the repair of the damaged
guns I brought with me and about an equal number found at Little Rock commenced at once .

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  k


dist: 0.8196150064468384

chunk: Stories are told through comic book @-@ like panels with animated character portraits , with characters 
speaking partially through voiced speech bubbles and partially through unvoiced text .The player progresses through
a series of linear missions , gradually unlocked as maps that can be freely scanned through and replayed as they 
are unlocked . The route to each story location on the map varies depending on an individual player 's approach : 
when one option is selected , the other is sealed off to the player .

Tuning ended, thresh value: 0.8196150064468384 = 0.82

**chatlogs**

In [6]:
splitter = SentenceSplitter(text=datasets['chatlogs'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 257/257 [00:13<00:00, 18.87it/s]


257 chunks formed

Values range: [-0.026845984160900116 ... 1.000000238418579]

Steps to tune: 8

--------------------------------

...

chunk: The city has a Zero Waste Strategy, which aims to have no waste sent to landfills by 2025.Zurich has an 
extensive recycling program, and residents are encouraged to separate their waste into different categories, 
including paper, glass, metal, and plastic. Organic waste is also collected separately and composted, which is then
used to fertilize public parks and gardens.\n\n9.

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  k


dist: 0.6262622475624084

chunk: Do you think they have anything with a shrunken head as a garnish?That would be a trip.", 'role': 'user'}, 
{'content': "I am not sure if the Viktor Wynd Museum of Curiosities serves cocktails with an actual shrunken head 
as a garnish. However, they might have some creative cocktails that incorporate a shrunken head replica or 
something similar as an element in the drink.

Tuning ended, thresh value: 0.6262622475624084 = 0.62

**finance**

In [7]:
splitter = SentenceSplitter(text=datasets['finance'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6320/6320 [06:03<00:00, 17.40it/s]


3716 chunks formed

Values range: [-0.044754236936569214 ... 1.0]

Steps to tune: 11

--------------------------------

...

chunk: cme group has no shares of preferred stock issued and outstanding.associated trading rights. members of cme,
cbot, nymex and comex own or lease trading rights which entitle them to access open outcry trading, discounts on 
trading fees and the right to vote on certain exchange matters as provided for by the rules of the particular 
exchange and cme group 2019s or the subsidiaries 2019 organizational documents.

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.7286220788955688

chunk: payment of dividends in the future, when, as and if authorized by our board of directors, would depend upon 
many factors, including our earnings and financial condition, restrictions under applicable law and our current and
future loan agreements, our debt service requirements, our capital expenditure requirements and other factors that 
our board of directors may deem relevant from time to time, including the potential determination to elect reit 
status.in addition, the loan agreement for our revolving credit facility and term loan contain covenants that 
generally restrict our ability to pay dividends unless certain financial covenants are satisfied. for more 
information about the restrictions under the loan agreement for the revolving credit facility and term loan, our 
notes indentures and the loan agreement related to our securitization, see item 7 of this annual report under the 
caption 201cmanagement 2019s discussion and analysis of financial condition and results of operations 2014liquidity
and capital resources 2014factors affecting sources of liquidity 201d and note 6 to our consolidated financial 
statements included in this annual report..

Tuning ended, thresh value: 0.7286220788955688 = 0.72

**pubmed**

In [8]:
splitter = SentenceSplitter(text=datasets['pubmed'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4458/4458 [04:05<00:00, 18.14it/s]


4432 chunks formed

Values range: [-0.1381773203611374 ... 1.000000238418579]

Steps to tune: 12

--------------------------------

...

chunk: First, the genetic map positions of all uncloned genetic mutants with visible phenotypes were checked using 
WormBase (http://www.wormbase.org, the Internet site for the genetics, genomics, and biology of C.elegans). Second,
we searched for clones near the defined map positions that, when fed to N2, rrf-3, or both, gave phenotypes 
corresponding to the phenotypes of the genetic mutants.

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.7016535997390747

chunk: falciparum IDC dataset, a similar computational approach was taken.We performed simple Fourier analysis, 
which allowed us to calculate both the apparent phase and frequency of expression for each gene during the IDC (see
Materials and Methods). The fast Fourier transform (FFT) maps a function in a time domain (the expression profile) 
into a frequency domain such that when the mapped function is plotted (the power spectra), sharp peaks appear at 
frequencies where there is intrinsic periodicity.

Tuning ended, thresh value: 0.7016535997390747 = 0.70

**state_of_the_union**

In [9]:
splitter = SentenceSplitter(text=datasets['state_of_the_union'])
splits = splitter()
res = tuner(splits, depth=DEPTH)
thresholds.append(res)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 658/658 [00:39<00:00, 16.84it/s]


657 chunks formed

Values range: [0.06096596270799637 ... 0.9999999403953552]

Steps to tune: 9

--------------------------------

...

chunk: I am here to- — tonight to show what I believe is the way forward, because I know how far we’ve come.Four 
years ago next week, before I came to office, the country was hit by the worst pandemic and the worst economic 
crisis in a century. Remember the fear, record losses?

Type 'k' to raise thresh, or 'j' - to lower it, then press 'Enter':  j


dist: 0.6557968258857727

chunk: In my career, I’ve been told I was too young.By the way, they didn’t let me on the Senate elevators for 
votes sometimes. They — not a joke.

Tuning ended, thresh value: 0.6557968258857727 = 0.65

#### Averaging thresh values 

The calculated average threshold value will be used for evaluation over the general dataset in the `evaluation.ipynb`notebook.

In [12]:
import numpy as np

print(len(thresholds), thresholds)
thresh = np.mean(thresholds)
print(f'avg. thresh: {thresh} = {str(thresh)[:4]}')

5 [0.8196150064468384, 0.6262622475624084, 0.7286220788955688, 0.7016535997390747, 0.6557968258857727]
avg. thresh: 0.7063899517059327 = 0.70
